In [5]:
import os
import cv2
import shutil

from pathlib import Path

# ====== CONFIG ======
pred_dir = "C:/Users/Welcome/phases_detection/runs/detect/val2/labels"
gt_dir = "C:/Users/Welcome/phases_detection/fine_tuning/labels/test"
image_dir = "C:/Users/Welcome/phases_detection/fine_tuning/images/test"
output_fp_image_dir = "C:/Users/Welcome/phases_detection/fine_tuning/false_positive"
iou_threshold = 0.5

os.makedirs(output_fp_image_dir, exist_ok=True)

# ====== Utility Functions ======
def load_boxes(label_path):
    boxes = []
    if not os.path.exists(label_path):
        return boxes
    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 5:
                cls, cx, cy, w, h = map(float, parts)
                x1 = cx - w / 2
                y1 = cy - h / 2
                x2 = cx + w / 2
                y2 = cy + h / 2
                boxes.append((x1, y1, x2, y2))
    return boxes

def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    if interArea == 0:
        return 0.0

    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

# ====== Main Loop ======
for pred_file in os.listdir(pred_dir):
    if not pred_file.endswith(".txt"):
        continue

    image_name = pred_file.replace(".txt", ".jpg")  # or .png
    image_path = os.path.join(image_dir, image_name)
    if not os.path.exists(image_path):
        continue

    pred_path = os.path.join(pred_dir, pred_file)
    gt_path = os.path.join(gt_dir, pred_file)

    pred_boxes = load_boxes(pred_path)
    gt_boxes = load_boxes(gt_path)

    # Check each predicted box
    false_positives = 0
    for pred_box in pred_boxes:
        matched = False
        for gt_box in gt_boxes:
            iou = compute_iou(pred_box, gt_box)
            if iou >= iou_threshold:
                matched = True
                break
        if not matched:
            false_positives += 1

    # Save image if there are FPs
    if false_positives > 0:
        shutil.copy(image_path, os.path.join(output_fp_image_dir, image_name))


In [7]:
import os
import cv2

# --- CONFIGURATION ---
labels_folder = "C:/Users/Welcome/phases_detection/runs/detect/val2/labels" # Folder with .txt files (YOLO format)
images_folder = r"C:\Users\Welcome\phases_detection\fine_tuning\false_positive"       # Folder with .jpg/.png files
output_folder = r"C:\Users\Welcome\phases_detection\fine_tuning\false_positive\with_box"    # Where annotated images will be saved

# Create output folder if not exist
os.makedirs(output_folder, exist_ok=True)

# Supported image extensions
image_exts = [".jpg", ".jpeg", ".png"]

# Process each label file
for label_file in os.listdir(labels_folder):
    if not label_file.endswith(".txt"):
        continue

    base_name = os.path.splitext(label_file)[0]

    # Find the image
    image_path = None
    for ext in image_exts:
        possible = os.path.join(images_folder, base_name + ext)
        if os.path.exists(possible):
            image_path = possible
            break

    if image_path is None:
        print(f"[!] No image found for label: {label_file}")
        continue

    # Load image
    img = cv2.imread(image_path)
    if img is None:
        print(f"[!] Failed to load image: {image_path}")
        continue
    h, w = img.shape[:2]

    # Read label and draw boxes
    with open(os.path.join(labels_folder, label_file), 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            _, cx, cy, bw, bh = map(float, parts)
            x1 = int((cx - bw / 2) * w)
            y1 = int((cy - bh / 2) * h)
            x2 = int((cx + bw / 2) * w)
            y2 = int((cy + bh / 2) * h)

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, "Object", (x1, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Save annotated image
    out_path = os.path.join(output_folder, os.path.basename(image_path))
    cv2.imwrite(out_path, img)
    print(f"[✓] Saved annotated: {out_path}")


[!] No image found for label: img_0.txt
[!] No image found for label: img_135.txt
[!] No image found for label: img_144.txt
[!] No image found for label: img_160.txt
[✓] Saved annotated: C:\Users\Welcome\phases_detection\fine_tuning\false_positive\with_box\img_171.jpg
[✓] Saved annotated: C:\Users\Welcome\phases_detection\fine_tuning\false_positive\with_box\img_173.jpg
[!] No image found for label: img_204.txt
[!] No image found for label: img_205.txt
[!] No image found for label: img_214.txt
[!] No image found for label: img_215.txt
[!] No image found for label: img_219.txt
[!] No image found for label: img_229.txt
[!] No image found for label: img_248.txt
[!] No image found for label: img_253.txt
[!] No image found for label: img_257.txt
[!] No image found for label: img_263.txt
[!] No image found for label: img_284.txt
[!] No image found for label: img_285.txt
[!] No image found for label: img_289.txt
[!] No image found for label: img_294.txt
[!] No image found for label: img_299.tx